ME17B158 - Omkar Nath

ME17B170 - Uma T.V.

A compressed version of program 3, that creates a pipeline of the PCA-SVM file, and saves the model using Pickle for future use

The data files used in the intermediary steps can be found at:<br>
https://drive.google.com/drive/folders/1KZQGxsVdWbNpCf0-0EmXhnsaKcOin4wA?usp=sharing

In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import sklearn 
import statsmodels.api as sm
from sklearn.cluster import k_means
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle

In [2]:
# Reading the previously created modified files
train=pd.read_csv('train_imputed.csv')
test=pd.read_csv('test.csv')

In [3]:
# Seprating into x and y
y_train=train['label']
x_train=train.drop('label',axis=1)

In [4]:
# Defining a Pipeline with all the models
pipe = Pipeline([('scaler', StandardScaler()),
                 ('pca',IncrementalPCA(n_components=120)),
                 ('svc', GridSearchCV(estimator = SVC(), param_grid = {"C": [5]}, scoring= 'accuracy', 
                                      cv = KFold(n_splits=3,shuffle=True,random_state=42),
                                      verbose = 1, return_train_score=True))])

In [5]:
# Fitting the model to the data
pipe.fit(x_train, y_train)  

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   39.0s finished


Pipeline(steps=[('scaler', StandardScaler()),
                ('pca', IncrementalPCA(n_components=120)),
                ('svc',
                 GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
                              estimator=SVC(), param_grid={'C': [5]},
                              return_train_score=True, scoring='accuracy',
                              verbose=1))])

In [6]:
# Saving the model pipeline
pickle.dump(pipe, open("Model.sav", 'wb'))

In [7]:
# Making predictions using the model
test_predict = pipe.predict(test)

In [8]:
# Converting Predictions to the required format
solution=pd.DataFrame()
indices=[]
for i in range(1000):
    indices.append('Sample_'+str(i+1)) 
solution['Expected']=test_predict
solution.index=indices
solution.index.name = "Id"

In [9]:
# Saving the File
solution.to_csv('Predictions.csv')